In [1]:
import pandas as pd
import random
from ast import literal_eval
import gurobipy as gb
import numpy as np
import networkx as nx
import time as tm
import warnings
warnings.filterwarnings('ignore')

In [2]:
def dataframe_to_list_without_nan(dataframe):
    output = list()
    for i in range(len(dataframe)):
        list1 = list()
        for j in range(len(dataframe.iloc[0,:])):
            if(pd.notnull(dataframe.iloc[i,j])):
                list1.append(dataframe.iloc[i,j])
            else:
                break
        output.append(list1)
    return output

def dataframe_to_list(dataframe):
    output = list()
    for i in range(len(dataframe)):
        output.append(dataframe.iloc[i,0])
    return output

Inizializzazione dataset voli

In [3]:
thy_flights = pd.read_csv('Big Airline Models/THY_data/THY_flights.csv').iloc[:,1:]

nid_index_dict = dict()
index_nid_dict = dict()
for i in range(len(thy_flights)):
    nid_index_dict[thy_flights.iloc[i,0]] = i
    index_nid_dict[i] = thy_flights.iloc[i,0]

In [4]:
flights_copies = list()
counter = 1001
for i in range(len(thy_flights)):
    list1 = list()
    list1.append(thy_flights.iloc[i,0])
    list1.append(counter)
    counter += 1
    list1.append(counter)
    counter += 1
    flights_copies.append(list1)

Creazione itinerari

In [5]:
I = dataframe_to_list_without_nan(pd.read_csv('Big Airline Models/THY_data/THY_itineraries.csv').iloc[:,1:])

In [6]:
FI = dataframe_to_list_without_nan(pd.read_csv('Big Airline Models/THY_data/THY_FI.csv').iloc[:,1:])

In [7]:
IR = dataframe_to_list_without_nan(pd.read_csv('Big Airline Models/THY_data/THY_IR.csv').iloc[:,1:])

In [8]:
D = dataframe_to_list(pd.read_csv('Big Airline Models/THY_data/THY_D.csv').iloc[:,1:])

In [9]:
Fare = dataframe_to_list(pd.read_csv('Big Airline Models/THY_data/THY_Fare.csv').iloc[:,1:])

In [10]:
routes = dataframe_to_list_without_nan(pd.read_csv('Big Airline Models/THY_data/THY_routes.csv').iloc[:,1:])

AIRCRAFT ROUTES

In [11]:
A = list()
for i in range(1,len(routes)+1):
    if (i<10):
        s = "00" + str(i)
    if (i>= 10 and i<100):
        s = "0" + str(i)
    if (i>=100):
        s = str(i)
    A.append("THY"+s)

In [12]:
def substitution_in_list(r,f,p):
    index = len(r)+1
    for i in range(len(r)):
        if (r[i] == f):
            index = i
            break
    r[index] = p
    return r

In [13]:
R2 = list()
for r in routes:
    list1 = list()
    list1.append(list(r))
    for f in r:
        for p in flights_copies[nid_index_dict[f]]:
            list2 = list(r)
            list2 = substitution_in_list(list2,f,p)
            if (list2 != list(r)):
                list1.append(list2)
    R2.append(list1)          

R = list()
counter = 0
for i in range(len(R2)):
    list1 = list()
    for j in range(len(R2[i])):
        list1.append(counter)
        counter+=1
    R.append(list1)
    
R3 = list()
for i in range(len(R2)):
    for j in range(len(R2[i])):
        R3.append(R2[i][j])

In [14]:
aircraft_types = thy_flights['aircraft_type'].unique()
aircraft_clusters = pd.read_csv('aircraftClustering.csv')
cluster_seat = pd.read_csv('aircraftSeats.csv')

In [15]:
aircrafts = pd.DataFrame(A, columns =['aircraft_ID'])

In [16]:
def getAircraftType(aircraft):
    return thy_flights[thy_flights['aircraft_id']==aircraft].iloc[0,13]

def get_number_seats(aircraft):
    aircraft_cluster = aircraft_clusters[aircraft_clusters['AircraftType']==aircraft]['AssignedAircraftType'].iloc[0]
    seats = cluster_seat[cluster_seat['Aircraft']==str(aircraft_cluster)].iloc[0,2]
    return seats

In [17]:
aircraft_types = list()
for a in A:
    aircraft_types.append(getAircraftType(a))
aircrafts['type'] = aircraft_types

aircraft_seats = list()
for t in aircrafts['type']:
    aircraft_seats.append(get_number_seats(t))
aircrafts['seats'] = aircraft_seats

**SETS**

In [18]:
F = thy_flights.iloc[:,0]
#A
#IC
FM = thy_flights[thy_flights.mandatory == 1].iloc[:,0]
FO = thy_flights[thy_flights.mandatory == 0].iloc[:,0]
#IR
#FIO
#R
P = flights_copies

In [19]:
IC = list()
for i in FI:
    if (len(i)==2):
        IC.append(i)
        
IC_it = list()
for i in I:
    if (len(i)==3):
        IC_it.append(i)
        
IC_N_it = list()
for i in range(len(I)):
    if (len(I[i])==3):
        IC_N_it.append(i)

In [20]:
def canBeRedirect(i, j):
    dest = I[i][-1]
    if (any((I[i][a] in I[j])and(dest in I[j])and(I[j].index(I[i][a])<I[j].index(dest))for a in range(len(I[i])-1))):
        return True
    return False    

In [21]:
FIO = list()
for i in range(len(FI)):
    list1 = list()
    for f in FI[i]:
        if(thy_flights.iloc[nid_index_dict[f],23] == 0):
            list1.append(f)
    FIO.append(list1)

**PARAMETERS**

In [22]:
def dateCoding(date):
    d = 0
    day = str(date)[8:10]
    if (day == '12'):
        d = 1
    h = str(date)[11:13]
    m = str(date)[14:16]
    return d*1440+int(h)*60 + int(m)

def calc_flight_cost(flight):
    dep = flight['sobt']
    arr = flight['sibt']
    dep = pd.to_datetime(str(dep))
    arr = pd.to_datetime(str(arr))
    return calc_minutes(arr-dep) * 2000/60
    
def calc_minutes(date):
    h = str(date)[7:9]
    m = str(date)[10:12]
    return int(m)+int(h)*60

def calc_route_cost(route):
    cost = 0
    for f in route:
        if (f > 10000):
            cost += calc_flight_cost(thy_flights.iloc[nid_index_dict[f], :])
        else:
            cost += calc_flight_cost(thy_flights.iloc[int((f - 1001) / 2), :])
    return cost

In [23]:
#D[i] the demand forecast for passenger itinerary i.
#DI[i] the maximum number of passengers carried in itinerary i.
#c[r] the operation and delay penalty cost of executing aircraft route r.
#Cap[a] the number of seats (capacity) in aircraft a.
Cap = aircrafts.iloc[:,2]
#Fare[i] the average fare for itinerary i.
#STD[f] the schedule time of departure of flight f
STD = thy_flights['sobt']
#MPT the minimum passenger connection time
MPT = 30
#MTT the minimum turnaround time.
MTT = 30
#FT[f] the block time of flight f.
#EPD[r][f] the expected propagated delays of flight f in route r.
#M the minimum possible connection time between flight f1, f2.
M = 30
#b[i][j] Recapture rate from itinerary i to itinerary j, bi = 1.
#dev[r][f] deviation from the STD of flight f in aircraft route r.
#θ1[f][p][r] 1 if flight f, copy p ∈ P is served in route r, and 0 otherwise. r,f,p f
#θ2[i][f] if itinerary i involves flight f, and 0 otherwise. i,f
#θ3[i][r] 1 if the component flights of connecting itinerary i are executed by the same aircraft in route r, and 0 otherwise. r,i 
#γ[i][f1][p1][f2][p2] 1 if itinerary i ∈ IC exists between flight f1, copy p1 and flight f2, copy p2. f1 p1,f2 p2

In [24]:
def existICbetween(f1,p1,f2,p2):
    for i in IC:
        if (((i[0] == f1)or(i[0]==p1)) and ((i[1] == f2)or(i[1] == p2))):
            return True
    return False

def fare_for_flight(flight_id):
    flight = thy_flights[thy_flights['nid']==flight_id].iloc[0,:]
    return flight['gcdistance'] / 10
    

def fare_for_itinerary(itinerary):
    fare = 0
    for f in itinerary:
        fare += fare_for_flight(f)
    return fare

def find_min_cap(itinerary):
    minimum = np.Inf
    for i in itinerary:
        cap = thy_flights[thy_flights['nid']==i].iloc[0,17]
        if (cap < minimum ):
            minimum = cap
    return minimum

def find_min_pax(itinerary):
    minimum = np.Inf
    for i in itinerary:
        pax = thy_flights[thy_flights['nid']==i].iloc[0,18]
        if (pax < minimum ):
            minimum = pax
    return minimum

In [25]:
DI = list()
for i in range(len(FI)):
    DI.append(find_min_cap(FI[i]))
    
c = list()
for i in range(len(R3)):
    c.append(calc_route_cost(R3[i]))
    
FT = list()
for i in range(len(F)):
    dep = pd.to_datetime(thy_flights['sobt']).iloc[i]
    arr = pd.to_datetime(thy_flights['sibt']).iloc[i]
    time = arr - dep
    FT.append(calc_minutes(time))

dev = list()
for r in range(len(R3)):
    list1 = list()
    for f in range(len(F)):
        if (P[f][0] in R3[r]):
            list1.append(random.randint(1,10))
        else:
            list1.append(0)
    dev.append(list1)
             
EPD = list()
for r in range(len(R3)):
    list1 = list()
    for f in range(len(R3[r])):
        delay_sum = 0
        for d in range(f):
            delay_sum += dev[r][f]
        list1.append(delay_sum)
    EPD.append(list1)
             
             
B = list()
for i in range(len(I)):
    list1 = np.zeros(len(I))
    rate_max = 0.4
    for j in IR[i]:
        if rate_max == 0:
            break
        else:
            r = random.randint(0, 5) / 100
            if rate_max > r:
                list1[int(j)] = r
                rate_max -= r
            else:
                list1[int(j)] = rate_max
                rate_max = 0
    B.append(list1)

Θ1 = list()
for i in range(len(thy_flights)):
    list1 = list()
    for p in P[i]:
        list2 = list()
        for j in range(len(R3)):
            if (p in R3[j]):
                list2.append(1)
            else:
                list2.append(0)
        list1.append(list2)
    Θ1.append(list1)
    
Θ2 = list()
for i in FI:
    list1 = np.zeros(len(thy_flights))
    for f in i:
        list1[nid_index_dict[f]] = 1
    Θ2.append(list1)

Θ3 = list()
for i in FI:
    list1 = list()
    for j in range(len(R3)):
        if (all(flight in R3[j] for flight in i)):
            list1.append(1)
        else:
            list1.append(0)
    Θ3.append(list1)

γ = list()
for i in FI:
    list1 = list()
    for f1 in i:
        list2 = list()
        for p1 in P[nid_index_dict[f1]]:
            list3 = list()
            for f2 in i:
                list4 = list()
                for p2 in P[nid_index_dict[f2]]:
                    if f1 == f2 or p1 == p2 or f1 == p2 or p2 == f2 or existICbetween(f1, p1, f2, p2):
                        list4.append(1)
                    else:
                        list4.append(0)
                list3.append(list4)
            list2.append(list3)
        list1.append(list2)        
    γ.append(list1)

**MODEL 1**

In [31]:
model1 = gb.Model()
model1.modelSense = gb.GRB.MAXIMIZE

# DECISION VARIABLES
# X[a,r] : 1 if aircraft a flies route r, and 0 otherwise
X = model1.addVars( [(a,r) for a in range(len(A)) for r in range(len(R[a]))], vtype=gb.GRB.BINARY)
# Y[f,p] : 1 if flight copy p of flight f is selected, and 0 otherwise
Y = model1.addVars( [(f,p) for f in range(len(F)) for p in range(len(P[f]))], vtype=gb.GRB.BINARY)
# H[i] : the number of passengers travelling in itinerary i
H = model1.addVars([i for i in range(len(I))], vtype=gb.GRB.INTEGER, lb = 0)

In [32]:
# (1) OBJECTIVE FUNCTION
model1.setObjective( 
     ((gb.quicksum( (Fare[i]*H[i]) for i in range(len(I)) )) - (gb.quicksum((gb.quicksum((c[r]*X[a,r])for r in range(len(R[a])))for a in range(len(A)) ) )  )))
# CONSTRAINTS
# (2) ensure that every flight is assigned to an aircraft
for f in range(len(F)):
    for p in range(len(P[f])):
        model1.addConstr(gb.quicksum((gb.quicksum(Θ1[f][p][R[a][r]]*X[a,r] for r in range(len(R[a])))) for a in range(len(A)) ) == Y[f,p] )
# (3) ensure that exactly one flight copy will be selected for each flight
for f in range(len(F)):
    model1.addConstr( gb.quicksum(Y[f,p] for p in range(len(P[f]))) == 1 )
# (4) ensure that every aircraft takes at most one route
for a in range(len(A)):
    model1.addConstr( gb.quicksum(X[a,r] for r in range(len(R[a]))) <= 1 )
# (5) limit the number of passengers taken on itineraries to the forecast demand 
for i in range(len(I)):
    model1.addConstr( H[i] <= D[i] )
# (6) limit the number of passengers taken on itineraries to the number of available aircraft seats
for f in range(len(F)):
    model1.addConstr( gb.quicksum((Θ2[i][f]*H[i]) for i in range(len(I))) <= 
                         gb.quicksum((Cap[a] * gb.quicksum((gb.quicksum((Θ1[f][p][R[a][r]]*X[a,r])for p in range(len(P[f]))))for r in range(len(R[a]))))for a in range(len(A)) ))  
# (7) impose nonlinear restrictions on one-stop itineraries in terms of feasible connection
# after re-timing (i.e. enough connection time is left for passenger connection)
for i in IC_N_it:
    for fm in range(len(F)):
        for fn in range(len(F)):
            if fm != fn and Θ2[i][fm] == 1 and Θ2[i][fn] == 1:
                model1.addConstr(H[i] <= D[i] * gb.quicksum(
                    (gb.quicksum(Y[fm, pm] * Y[fn, pn] for pn in range(len(P[fn])))) for pm in range(len(P[fm]))))

In [33]:
model1.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-8210Y CPU @ 1.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 24354 rows, 24354 columns and 95778 nonzeros
Model fingerprint: 0x0e8b1edb
Model has 34320 quadratic constraints
Variable types: 0 continuous, 24354 integer (6620 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  QMatrix range    [9e+01, 3e+02]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [7e+00, 7e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Presolve removed 21379 rows and 3623 columns
Presolve time: 0.99s
Presolved: 329015 rows, 175171 columns, 839588 nonzeros
Variable types: 0 continuous, 175171 integer (157471 binary)
Found heuristic solution: objective 1983163.1333

Deterministic concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Root barrier log...

Ordering time: 2.21s

Barrier statistics

**MODEL 1 LINEAR**

In [26]:
model1 = gb.Model()
model1.modelSense = gb.GRB.MAXIMIZE

# DECISION VARIABLES
# X[a,r] : 1 if aircraft a flies route r, and 0 otherwise
X = model1.addVars( [(a,r) for a in range(len(A)) for r in range(len(R[a]))], vtype=gb.GRB.BINARY)
# Y[f,p] : 1 if flight copy p of flight f is selected, and 0 otherwise
Y = model1.addVars( [(f,p) for f in range(len(F)) for p in range(len(P[f]))], vtype=gb.GRB.BINARY)
# W[f1,p1,f2,p2] : 1 if flight copy pair (f1p1,f2,p2)is selected, 0 otherwise
W = model1.addVars( [(f1,p1,f2,p2) for f1 in range(len(F)) for p1 in range(len(P[f1])) 
                         for f2 in range(len(F)) for p2 in range(len(P[f2]))], vtype=gb.GRB.BINARY)
# H[i] : the number of passengers travelling in itinerary i
H = model1.addVars([i for i in range(len(I))], vtype=gb.GRB.INTEGER, lb = 0)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-26


In [27]:
# (1) OBJECTIVE FUNCTION
model1.setObjective( 
     ((gb.quicksum( (Fare[i]*H[i]) for i in range(len(I)) )) - (gb.quicksum((gb.quicksum((c[r]*X[a,r])for r in range(len(R[a])))for a in range(len(A)) ) )  )))
# CONSTRAINTS
# (2) ensure that every flight is assigned to an aircraft
for f in range(len(F)):
    for p in range(len(P[f])):
        model1.addConstr(gb.quicksum((gb.quicksum(Θ1[f][p][R[a][r]]*X[a,r] for r in range(len(R[a])))) for a in range(len(A)) ) == Y[f,p] )
# (3) ensure that exactly one flight copy will be selected for each flight
for f in range(len(F)):
    model1.addConstr( gb.quicksum(Y[f,p] for p in range(len(P[f]))) == 1 )
# (4) ensure that every aircraft takes at most one route
for a in range(len(A)):
    model1.addConstr( gb.quicksum(X[a,r] for r in range(len(R[a]))) <= 1 )
# (5) limit the number of passengers taken on itineraries to the forecast demand 
for i in range(len(I)):
    model1.addConstr( H[i] <= D[i] )
# (6) limit the number of passengers taken on itineraries to the number of available aircraft seats
for f in range(len(F)):
    model1.addConstr( gb.quicksum((Θ2[i][f]*H[i]) for i in range(len(I))) <= 
                         gb.quicksum((Cap[a] * gb.quicksum((gb.quicksum((Θ1[f][p][R[a][r]]*X[a,r])for p in range(len(P[f]))))for r in range(len(R[a]))))for a in range(len(A)) ))  
# (11)-(14) linearize constraint 7
#(11)
for i in range(len(FI)):
    for fm in FI[i]:
        for fn in FI[i]:
            if(Θ2[i][nid_index_dict[fm]] == 1 and Θ2[i][nid_index_dict[fn]] == 1):
                model1.addConstr( H[i] <= D[i]*gb.quicksum((gb.quicksum(γ[i][FI[i].index(fm)][pm][FI[i].index(fn)][pn]*W[nid_index_dict[fm],pm,nid_index_dict[fn],pn] for pn in range(len(P[nid_index_dict[fn]])))for pm in range(len(P[nid_index_dict[fm]]))) ))
#(12)
for i in range(len(I)):
    for fm in range(len(FI[i])):
        for fn in range(len(FI[i])):
            for pm in range(len(P[fm])):
                for pn in range(len(P[fn])):
                    model1.addConstr( W[fm,pm,fn,pn] <= Y[fm,pm] )
#(13)
for i in range(len(I)):
    for fm in range(len(FI[i])):
        for fn in range(len(FI[i])):
            for pm in range(len(P[fm])):
                for pn in range(len(P[fn])):
                    model1.addConstr( W[fm,pm,fn,pn] <= Y[fn,pn] )
#(14)
for i in range(len(I)):
    for fm in range(len(FI[i])):
        for fn in range(len(FI[i])):
            for pm in range(len(P[fm])):
                for pn in range(len(P[fn])):
                    model1.addConstr( W[fm,pm,fn,pn] >= Y[fm,pm]+Y[fn,pn]-1 )

In [28]:
model1.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-8210Y CPU @ 1.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 1962346 rows, 14632038 columns and 4940758 nonzeros
Model fingerprint: 0x759ac03f
Variable types: 0 continuous, 14632038 integer (14614304 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [7e+00, 7e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Presolve removed 89659 rows and 14610204 columns (presolve time = 5s) ...
Presolve removed 1958341 rows and 14611484 columns (presolve time = 13s) ...
Presolve removed 1958341 rows and 14611484 columns (presolve time = 15s) ...
Presolve removed 1959973 rows and 14614063 columns (presolve time = 20s) ...
Presolve removed 1961413 rows and 14614322 columns (presolve time = 25s) ...
Presolve removed 1961469 rows and 14614359 columns (presolve time = 30s) ...
Presolve removed 196

**MODEL 2**

In [ ]:
model2 = gb.Model()
model2.modelSense = gb.GRB.MAXIMIZE

# X[a,r] : 1 if aircraft a flies route r, and 0 otherwise
X = model2.addVars([(a, r) for a in range(len(A)) for r in range(len(R[a]))], vtype=gb.GRB.BINARY)
# Y[f,p] : 1 if flight copy p of flight f is selected, and 0 otherwise
Y = model2.addVars([(f, p) for f in range(len(F)) for p in range(len(P[f]))], vtype=gb.GRB.BINARY)
# Z[i] : 1 if itinerary i is selected for inclusion, and 0 otherwise
Z = model2.addVars([i for i in range(len(I))], vtype=gb.GRB.BINARY)
# H[i] : the number of passengers travelling in itinerary i
H = model2.addVars([i for i in range(len(I))], vtype=gb.GRB.INTEGER, lb=0)
# T[i,j] : the number of passengers requesting itinerary i that the airline attempts to redirect to itinerary j
T = model2.addVars([(i, j) for i in range(len(I)) for j in range(len(I))], vtype=gb.GRB.INTEGER, lb=0)
# EDT[f] : the expected departure time of flight f under expected propagated delays
EDT = model2.addVars([f for f in range(len(F))], vtype=gb.GRB.INTEGER, lb=0)
# TF[i] : 1 if the two connecting flights of itinerary i are operated by the same aircraft (i.e. through flight), and 0 otherwise
TF = model2.addVars([i for i in range(len(IC))], vtype=gb.GRB.BINARY)

In [ ]:
# (1) OBJECTIVE FUNCTION
model2.setObjective(
    ((gb.quicksum((Fare[i] * H[i]) for i in range(len(I)))) - (
        gb.quicksum((gb.quicksum((C[r] * X[a, r]) for r in range(len(R[a]))) for a in range(len(A)))))))

# CONSTRAINTS
# (2) ensure that every flight is assigned to an aircraft
for f in range(len(F)):
    for p in range(len(P[f])):
        model2.addConstr(
            gb.quicksum((gb.quicksum(Θ1[f][p][R[a][r]] * X[a, r] for r in range(len(R[a])))) for a in range(len(A))) ==
            Y[f, p])
# (4) ensure that every aircraft takes at most one route
for a in range(len(A)):
    model2.addConstr(gb.quicksum(X[a, r] for r in range(len(R[a]))) <= 1)
# (17) flight cover constraint for mandatory flights
for f in FM:
    model2.addConstr(
        gb.quicksum(Y[airline.nid_index_dict[f], p] for p in range(len(P[airline.nid_index_dict[f]]))) == 1)
# (18) flight cover constraint for optional flights
for f in FO:
    model2.addConstr(
        gb.quicksum(Y[airline.nid_index_dict[f], p] for p in range(len(P[airline.nid_index_dict[f]]))) <= 1)
# (19) computation of expected flight departure time under propagated delays and re-timing decisions
for f in range(len(F)):
    model2.addConstr(EDT[f] == dateCoding(STD[f]) +
                     gb.quicksum((gb.quicksum((dev[r][f] * X[a, r]) for r in range(len(R[a])))) for a in range(len(A)))
                     )
# (20) restrict the number of passengers spilled from an itinerary to be no more than the demand of that itinerary
for i in range(len(I)):
    model2.addConstr(H[i] <= D[i] - gb.quicksum(((T[i, j] - B[i][j] * T[j, i])) for j in IR[i]))
# (21) ensure that if an optional flight is not executed, all itineraries containing this flight will not be served as well
for i in range(len(I)):
    for f in FIO[i]:
        model2.addConstr(
            Z[i] <= gb.quicksum((Y[airline.nid_index_dict[f], p]) for p in range(len(P[airline.nid_index_dict[f]]))))
# (22)
for i in range(len(IC)):
    model2.addConstr(TF[i] == gb.quicksum(
        (gb.quicksum((Θ3[i][R[a][r]] * X[a, r]) for r in range(len(R[a])))) for a in range(len(A))))
# (23) require that the connection time of every one-stop itinerary must be no less than the minimum passenger connection time
for i in range(len(IC)):
    f1 = airline.nid_index_dict[IC[i][0]]
    f2 = airline.nid_index_dict[IC[i][1]]
    model2.addConstr(EDT[f2] - EDT[f1] - FT[f1] >=
                     (-MPT + MTT) * TF[i] + MPT * Z[IC_it[i]] - M * (1 - Z[IC_it[i]]))
# (24) ensure that the number of passengers flown does not exceed the aircraft capacity
for f in range(len(F)):
    model2.addConstr(gb.quicksum((Θ2[i][f] * H[i]) for i in range(len(I))) <=
                     gb.quicksum((Cap[a] * gb.quicksum(
                         (gb.quicksum((Θ1[f][p][R[a][r]] * X[a, r]) for p in range(len(P[f])))) for r in
                         range(len(R[a])))) for a in range(len(A))))
# (25) assert that no passenger is served on an inactivated itinerary
for i in range(len(I)):
    model2.addConstr(H[i] <= DI[i] * Z[i])
# (30) number of passengers redirect from itinerary i to j must be less or equal than the demand forecast for passenger itinerary i
for i in range(len(I)):
    for j in range(len(IR[i])):
        model2.addConstr(T[i, j] <= D[i])

In [ ]:
model2.optimize()

**VNS**

In [38]:
class VNS:
    def __init__(self, airline):
        self.airline = airline
        self.feasibleSolutionSearcher = FeasibleSolutionSearcher(airline)
        self.evaluator = Evaluator(airline)
        self.neighborhood = Neighborhood(thy_flights)
        self.current_solution = self.feasibleSolutionSearcher.search()
        self.neighborhood_func = self.neighborhood.neighborhood
        self.objective_value = self.evaluator.evaluate(self.current_solution)
        
    def search(self,N):
        k = 1
        while k <= N:
            neighbors = self.neighborhood_func(self.current_solution)
            neighbor = random.choice(neighbors)
            if neighbor is not None:
                obj = self.evaluator.evaluate(neighbor)
                if self.objective_value >= obj:
                    k += 1
                else:
                    self.objective_value,self.current_solution = obj,neighbor
        return self.current_solution,self.objective_value

In [37]:
class FeasibleSolutionSearcher:
    def __init__(self, airline):
        self.airline = airline
        self.routes_graph = self.create_routes_graph()

    def search(self):
        D = self.routes_graph
        edge_index_dict = dict()
        index_edge_dict = dict()
        D_edges = list()
        counter = 0
        for e in D.edges():
            D_edges.append(list([e[0], e[1]]))
            edge_index_dict[e] = counter
            index_edge_dict[counter] = e
            counter += 1
        A0 = self.createA0(D)
        ip = gb.Model()
        ip.Params.LogToConsole = 0
        X = ip.addVars([a for a in range(len(D.edges()))], vtype=gb.GRB.BINARY)
        # (1)
        for v in D.nodes():
            if D.in_degree(v) != 0 and D.out_degree(v) != 0:
                ip.addConstr(gb.quicksum(X[edge_index_dict[(a, v)]] for a in D.predecessors(v)) == gb.quicksum(
                    X[edge_index_dict[(v, a)]] for a in D.successors(v)))
        # (2)
        for v in D.nodes():
            if D.in_degree(v) != 0:
                ip.addConstr(gb.quicksum(X[edge_index_dict[(a, v)]] for a in D.predecessors(v)) == 1)
        # (3)
        ip.addConstr(gb.quicksum(X[edge_index_dict[a]] for a in A0) <= len(D_edges))
        ip.optimize()
        D2 = nx.Graph()
        for i in range(len(X)):
            if X[i].x == 1:
                D2.add_edge(D_edges[i][0], D_edges[i][1])
        feasible_solution = list()
        for i in nx.connected_components(D2):
            feasible_solution.append(list(i))
        return feasible_solution

    def createA0(self, D):
        instant = pd.to_datetime('2014-09-12 ' + str(23) + ':05:00')
        A0 = list()
        for e in D.edges():
            if ((pd.to_datetime(thy_flights[thy_flights.nid == e[0]].iloc[0, 11]) <= instant)
                    and (pd.to_datetime(thy_flights[thy_flights.nid == e[1]].iloc[0, 11]) > instant)):
                A0.append(e)
        return A0

    def create_routes_graph(self):
        D = nx.DiGraph()
        for i in range(len(routes)):
            for j in range(len(routes[i]) - 1):
                D.add_edge(routes[i][j], routes[i][j + 1])
        return D


In [31]:
def get_cap_flight(flights, flight_nid):
    return flights[flights.nid == flight_nid].iloc[0,17]

In [35]:
class Evaluator:
    def __init__(self, airline):
        self.airline = airline

    def evaluate(self, P):
        flights_selected = list()
        for r in P:
            for f in r:
                if not (f in flights_selected):
                    flights_selected.append(f)
        itineraries_selected = list()
        for i in range(len(FI)):
            if all(f in flights_selected for f in FI[i]):
                itineraries_selected.append(i)
        flight_index_ev_dict = dict()
        index_flight_ev_dict = dict()
        c = 0
        for f in flights_selected:
            flight_index_ev_dict[f] = c
            index_flight_ev_dict[c] = f
            c += 1
        itinerary_index_ev_dict = dict()
        index_itinerary_ev_dict = dict()
        c = 0
        for i in itineraries_selected:
            itinerary_index_ev_dict[i] = c
            index_itinerary_ev_dict[c] = i
            c += 1
        fare = list()
        for i in itineraries_selected:
            fare.append(Fare[i])
        direct_it = pd.DataFrame(columns=['n_dit', 'fare_d'])
        onestop_it = pd.DataFrame(columns=['n_osit', 'fare_os'])
        for i in range(len(fare)):
            if len(FI[itineraries_selected[i]]) == 1:
                direct_it.loc[len(direct_it)] = [i, fare[i]]
            elif len(FI[itineraries_selected[i]]) == 2:
                onestop_it.loc[len(onestop_it)] = [i, fare[i]]
        direct_it_ord = direct_it.sort_values('fare_d', ascending=False)
        onestop_it_ord = onestop_it.sort_values('fare_os', ascending=False)
        di = list()
        for i in itineraries_selected:
            di.append(D[i])
        H = np.zeros(len(itineraries_selected))
        cap = list()
        for f in flights_selected:
            cap.append(get_cap_flight(thy_flights,f))
        remain = cap
        # step1
        for i in range(len(onestop_it_ord)):
            f1 = FI[itineraries_selected[int(onestop_it_ord.iloc[i, 0])]][0]
            f2 = FI[itineraries_selected[int(onestop_it_ord.iloc[i, 0])]][1]
            flow = min(remain[flight_index_ev_dict[f1]], remain[flight_index_ev_dict[f2]],
                       di[int(onestop_it_ord.iloc[i, 0])])
            H[int(onestop_it_ord.iloc[i, 0])] = flow
            remain[flight_index_ev_dict[f1]] -= flow
            remain[flight_index_ev_dict[f2]] -= flow
        # step2
        for i in range(len(direct_it_ord)):
            f = FI[itineraries_selected[int(direct_it_ord.iloc[i, 0])]][0]
            flow = min(di[int(direct_it_ord.iloc[i, 0])], remain[flight_index_ev_dict[f]])
            H[int(direct_it_ord.iloc[i, 0])] = flow
            remain[flight_index_ev_dict[f]] -= flow
        costs = list()
        for r in P:
            costs.append(calc_route_cost(r))
        cost = gb.quicksum(costs[r] for r in range(len(costs))).getValue()
        revenue = gb.quicksum(fare[i] * H[i] for i in range(len(fare))).getValue()
        return revenue - cost

In [36]:
class Neighborhood:
    def __init__(self, flights):
        self.flights = flights

    def neighborhood(self, P):
        neighbors = []
        neighbors.append(self.cross(P))
        neighbors.append(self.insert(P))
        neighbors.append(self.swap(P))
        neighbors.append(self.delete(P))
        return neighbors

    def cross(self, P):
        n1 = random.randint(0, len(P) - 1)
        for n2 in range(len(P)):
            if n1 != n2:
                for u in range(1, len(P[n1])):
                    for v in range(1, len(P[n2])):
                        if self.con(P[n1][u - 1], P[n2][v]) and self.con(P[n2][v - 1], P[n1][u]):
                            P2 = P
                            P2[n1][u] = P[n2][v]
                            P2[n2][v] = P[n1][u]
                            return P2

    def insert(self, P):
        n1 = random.randint(0, len(P) - 1)
        for n2 in range(len(P)):
            if n1 != n2:
                for u in range(1, len(P[n1])):
                    for v in range(1, len(P[n2])):
                        for w in range(0, len(P[n2]) - 1):
                            if (v != w and self.con(P[n1][u - 1], P[n2][v]) and self.con(P[n2][w], P[n1][u])
                                    and self.arr(P[n2][v - 1]) == self.dep(P[n2][w + 1])):
                                P2 = P
                                x = P[n2][v:w + 1]
                                for i in range(len(x)):
                                    P2[n2].remove(x[i])
                                for i in range(len(x)):
                                    P2[n1].insert(u + i, x[i])
                                return P2

    def swap(self, P):
        n1 = random.randint(0, len(P) - 1)
        for n2 in range(len(P)):
            if n1 != n2:
                for u in range(1, len(P[n1]) - 1):
                    for v in range(1, len(P[n2]) - 1):
                        if (self.con(P[n1][u - 1], P[n2][v]) and self.con(P[n2][v], P[n1][u + 1])
                                and self.con(P[n2][v - 1], P[n1][u]) and self.con(P[n1][u], P[n2][v + 1])):
                            P2 = P
                            P2[n1][u] = P[n2][v]
                            P2[n2][v] = P[n1][u]
                            return P2

    def delete(self, P):
        n1 = random.randint(0, len(P) - 1)
        for u in range(len(P[n1])):
            if self.is_optional(P[n1][u]):
                for n2 in range(len(P)):
                    if n1 != n2:
                        for v1 in range(len(P[n2]) - 1):
                            if self.dep(P[n1][u]) == self.dep(P[n2][v1]):
                                for v2 in range(v1, len(P[n2])):
                                    if self.arr(P[n1][u]) == self.dep(P[n2][v2]):
                                        P2 = P
                                        P2[n1].remove(P[n1][u])
                                        for i in range(v1, v2 + 1):
                                            P2[n1].insert(u + (i - v1), P[n2][i])
                                        return P2

    def dep(self, f):
        return self.flights[self.flights.nid == f].iloc[0, 7]

    def arr(self, f):
        return self.flights[self.flights.nid == f].iloc[0, 8]

    def con(self, f1, f2):
        if self.arr(f1) == self.dep(f2) and pd.to_datetime(self.flights.iloc[nid_index_dict[f1],11])-pd.to_datetime(self.flights.iloc[nid_index_dict[f2],12]) > pd.Timedelta(30, "m"):
            return True
        return False

    def is_optional(self, f):
        if self.flights[self.flights.nid == f].iloc[0, 23] == 0:
            return True
        return False

In [39]:
vns = VNS('THY')
start = tm.perf_counter()
sol, obj_value = vns.search(10)
end = tm.perf_counter()
print('Revenue(objective function value): ' + str("{:e}".format(obj_value)))
print('Task time: ' + str(end - start) + ' s')
print('Solution aircraft routing problem: ')
print(sol)

Revenue(objective function value): 8.416313e+06
Task time: 867.7547237360004 s
Solution aircraft routing problem: 
[[60841.0, 33718.0, 37422], [33719.0, 49183], [47440, 33723.0], [42849, 55651.0, 67004.0, 33725.0], [48664, 42099.0, 38571.0, 33731.0], [33741.0, 67125.0, 39029, 47421.0], [46818.0, 58004, 33750.0, 37724.0], [66905, 60386.0, 52692.0, 33761.0], [38355, 33781.0, 59595.0], [39305, 33796.0, 60393.0, 48884.0], [49177, 33803.0, 42566.0, 37847.0], [38131, 47557.0, 33806.0, 65941.0], [39610, 33950.0, 50586.0], [38744, 57865.0, 48379.0, 34039.0], [36674, 46378.0, 52523.0, 62282.0, 34064.0, 57369.0, 41404.0], [44208, 34089.0, 51765.0, 59901.0], [52353, 46055.0, 66152.0, 52692.0, 34109.0], [49826, 40591.0, 35956.0, 59605.0, 34270.0], [36224, 34354.0, 48118.0], [66656, 36322.0, 60037.0, 46061.0, 34381.0, 51886.0, 42097.0], [45516, 34382.0, 51982.0, 59764.0, 66836.0, 38873.0], [56813.0, 67085, 45518.0, 34420.0, 37403.0], [59588, 44041.0, 54825.0, 48529.0, 35476.0, 34421.0, 66687.0], [5